In [1]:

import pandas as pd
import numpy as np
from numpy import random
from sklearn.model_selection import train_test_split
import ast
import os
from PIL import Image
import yaml

# 일부 에러 데이터 정규화

In [2]:
# import pandas as pd

# def modify_csv(input_file, output_file, column_name, new_value, first_zero):
#     # CSV 파일 읽기
#     df = pd.read_csv(input_file)

#     # 조건 설정: 첫 번째 0이 나오기 전까지만
#     row_condition = df.index < first_zero_frame_hd

#     # 조건에 해당하는 값 수정
#     df.loc[row_condition, column_name] = new_value

#     # 수정된 데이터 저장
#     df.to_csv(output_file, index=False)

# # 함수 사용 예시
# # modify_csv(
# #     "C:\\Users\\dawoo\\desktop\\sf7\\coding\\pytorch\\project\\data\\data\\output\\frame\\frame_hd_data.csv", 
# #     "./data/data/output/frame/frame_hd_data_modify.csv", 
# #     "bboxes", 
# #     "[]", 
# #     first_zero_frame_hd
# # )

# # modify_csv(
# #     "./data/data/output/frame/frame_sealf_data.csv", 
# #     "./data/data/output/frame/frame_sealf_data_modify.csv", 
# #     "bboxes", 
# #     "[]", 
# #     first_zero_frame_sealf
# # )

# # modify_csv(
# #     "./data/data/output/frame/frame_seamf_data.csv", 
# #     "./data/data/output/frame/frame_seamf_data_modify.csv", 
# #     "bboxes", 
# #     "[]", 
# #     first_zero_frame_seamf
# # )


# 데이터 전처리
- 양품 : 75 * 4 = 300
- 불량품 : 300 * 4 = 1200 랜덤 샘플링

In [3]:
from sklearn.model_selection import train_test_split
import pandas as pd

# CSV 파일 로드 함수
def load_data_with_pandas(file_path):
    data = pd.read_csv(file_path)
    return data

# 데이터 로드
data_bumper = load_data_with_pandas('./data/data/output/bumper/bumper_data.csv')
data_door_ed = load_data_with_pandas('./data/data/output/door/door_ed_data.csv')
data_door_scratch = load_data_with_pandas('./data/data/output/door/door_scratch_data.csv')
data_fender = load_data_with_pandas('./data/data/output/fender/fender_ed_data.csv')
data_frame_ed = load_data_with_pandas('./data/data/output/frame/frame_ed_data.csv')
data_frame_hd = load_data_with_pandas('./data/data/output/frame/frame_hd_data_modify.csv')
data_frame_sealf = load_data_with_pandas('./data/data/output/frame/frame_sealf_data_modify.csv')
data_frame_seamf = load_data_with_pandas('./data/data/output/frame/frame_seamf_data_modify.csv')


# 0이 처음 출력되는 index 값 추출
first_zero_bumper = data_bumper[data_bumper['quality'] == 0].index[0]

first_zero_door_ed = data_door_ed[data_door_ed['quality'] == 0].index[0]
first_zero_door_scratch = data_door_scratch[data_door_scratch['quality'] == 0].index[0]

first_zero_fender = data_fender[data_fender['quality'] == 0].index[0]

first_zero_frame_ed = data_frame_ed[data_frame_ed['quality'] == 0].index[0]
first_zero_frame_hd = data_frame_hd[data_frame_hd['quality'] == 0].index[0]
first_zero_frame_sealf = data_frame_sealf[data_frame_sealf['quality'] == 0].index[0]
first_zero_frame_seamf = data_frame_seamf[data_frame_seamf['quality'] == 0].index[0]



### index을 통해서 양품과 불량품의 위치를 구분

In [4]:
# print(f"bumper 양품 : {len(data_bumper.iloc[:first_zero_bumper])}")
# print(f"bumper 불량품 : {len(data_bumper.iloc[first_zero_bumper:])}")

# print(f"door_ed 양품 : {len(data_door_ed.iloc[:first_zero_door_ed])}")
# print(f"door_ed 불양품 : {len(data_door_ed.iloc[first_zero_door_ed:])}")
# print(f"door_scratch 양품 : {len(data_door_scratch.iloc[:first_zero_door_scratch])}")
# print(f"door_scratch 불양품 : {len(data_door_scratch.iloc[first_zero_door_scratch:])}")

# print(f"data_fender 양품 : {len(data_fender.iloc[:first_zero_fender])}")
# print(f"data_fender 불량품 : {len(data_fender.iloc[first_zero_fender:])}")

# print(f"data_frame_ed 양품 : {len(data_frame_ed.iloc[:first_zero_frame_ed])}")
# print(f"data_frame_ed 불양품 : {len(data_frame_ed.iloc[first_zero_frame_ed:])}")
# print(f"data_frame_hd 양품 : {len(data_frame_hd.iloc[:first_zero_frame_hd])}")
# print(f"data_frame_hd 불양품 : {len(data_frame_hd.iloc[first_zero_frame_hd:])}")
# print(f"data_frame_sealf 양품 : {len(data_frame_sealf.iloc[:first_zero_frame_sealf])}")
# print(f"data_frame_sealf 불양품 : {len(data_frame_sealf.iloc[first_zero_frame_sealf:])}")
# print(f"data_frame_seamf 양품 : {len(data_frame_seamf.iloc[:first_zero_frame_seamf])}")
# print(f"data_frame_seamf 불양품 : {len(data_frame_seamf.iloc[first_zero_frame_seamf:])}")



### 양품 데이터 추출

In [5]:
import pandas as pd
data_door = pd.concat([data_door_ed.iloc[:first_zero_door_ed],
                      data_door_scratch.iloc[:first_zero_door_scratch]], ignore_index=True)
data_frame = pd.concat([data_frame_ed.iloc[:first_zero_frame_ed],
                        data_frame_hd.iloc[:first_zero_frame_hd], data_frame_sealf.iloc[:first_zero_frame_sealf], data_frame_seamf.iloc[:first_zero_frame_seamf]], ignore_index=True)

bumper_before_zero = data_bumper.iloc[:first_zero_bumper]
fender_before_zero = data_fender.iloc[:first_zero_fender]


# 75개를 무작위로 추출 (복원 추출, random_state 설정)
bumper_ok_random_sample = bumper_before_zero.sample(
    n=min(75, len(bumper_before_zero)), replace=False, random_state=55)
fender_ok_random_sample = fender_before_zero.sample(
    n=min(75, len(fender_before_zero)), replace=False, random_state=55)
door_ok_random_sample = data_door.sample(n=75, replace=False)
frame_ok_random_sample = data_frame.sample(n=75, replace=False)
# 결과 출력
# print(bumper_ok_random_sample)
# print(len(bumper_ok_random_sample))

# print(fender_ok_random_sample)
# print(len(fender_ok_random_sample))

# print(door_ok_random_sample)
# print(len(door_ok_random_sample))

# print(frame_ok_random_sample)
# print(len(frame_ok_random_sample))

ok_data = pd.concat([bumper_ok_random_sample, fender_ok_random_sample,
                    door_ok_random_sample, frame_ok_random_sample], ignore_index=True, axis=0)

# ok_data

### 불량품 데이터 추출

In [6]:
import pandas as pd
data_fault_bumper = data_bumper.iloc[first_zero_bumper:]
data_fault_door = pd.concat([data_door_ed.iloc[first_zero_door_ed:],
                             data_door_scratch.iloc[first_zero_door_scratch:]], ignore_index=True)
data_fault_fender = data_fender.iloc[first_zero_fender:]
data_fault_frame = pd.concat([data_frame_ed.iloc[first_zero_frame_ed:],
                              data_frame_hd.iloc[first_zero_frame_hd:], data_frame_sealf.iloc[first_zero_frame_sealf:], data_frame_seamf.iloc[first_zero_frame_seamf:]], ignore_index=True)


# 300개를 무작위로 추출 (복원 추출, random_state 설정)
bumper_fault_random_sample = data_fault_bumper.sample(n=300, replace=False)
fender_fault_random_sample = data_fault_fender.sample(n=300, replace=False)
door_fault_random_sample = data_fault_door.sample(n=300, replace=False)
frame_fault_random_sample = data_fault_frame.sample(n=300, replace=False)
# 결과 출력
# print(bumper_fault_random_sample)
# print(len(bumper_fault_random_sample))

# print(fender_fault_random_sample)
# print(len(fender_fault_random_sample))

# print(door_fault_random_sample)
# print(len(door_fault_random_sample))

# print(frame_fault_random_sample)
# print(len(frame_fault_random_sample))

fault_data = pd.concat([bumper_fault_random_sample, fender_fault_random_sample,
                        door_fault_random_sample, frame_fault_random_sample], ignore_index=True, axis=0)

# fault_data

In [7]:
from sklearn.model_selection import train_test_split
import pandas as pd

# y에는 라벨, x에는 나머지 특성 정보
y_ok = ok_data['quality']
y_fault = fault_data['quality']

# Stratified Split: 학습 데이터와 검증 데이터 비율을 80:20으로 설정, 클래스 비율을 유지
train_data_ok, test_data_ok = train_test_split(
    ok_data, test_size=0.2, stratify=y_ok, random_state=60
)
train_data_fault, test_data_fault = train_test_split(
    fault_data, test_size=0.2, stratify=y_fault, random_state=60
)

# 학습 데이터에서 다시 80:20 비율로 나누어 검증 데이터 분리
train_data_80_ok, valid_data_ok = train_test_split(
    train_data_ok, test_size=0.2, stratify=train_data_ok['quality'], random_state=60)
train_data_80_fault, valid_data_fault = train_test_split(
    train_data_fault, test_size=0.2, stratify=train_data_fault['quality'], random_state=60)

# CSV로 저장
train_data_80_ok.to_csv('./data/train_data_80_ok.csv', index=False)
train_data_80_fault.to_csv('./data/train_data_80_fault.csv', index=False)
test_data_ok.to_csv('./data/test_data_ok.csv', index=False)
test_data_fault.to_csv('./data/test_data_fault.csv', index=False)
valid_data_ok.to_csv('./data/valid_data_ok.csv', index=False)
valid_data_fault.to_csv('./data/valid_data_fault.csv', index=False)

# 결과 출력
print(f"y_ok 총 데이터 개수: {len(y_ok)}")
print(f"y_fault 총 데이터 개수: {len(y_fault)}")
print(f"train_data_80_ok 학습 데이터 개수: {len(train_data_80_ok)}")
print(f"train_data_80_fault 학습 데이터 개수: {len(train_data_80_fault)}")
print(f"test_data_ok 테스트 데이터 개수: {len(test_data_ok)}")
print(f"test_data_fault 테스트 데이터 개수: {len(test_data_fault)}")
print(f"valid_data_ok 검증 데이터 개수: {len(valid_data_ok)}")
print(f"valid_data_fault 검증 데이터 개수: {len(valid_data_fault)}")


y_ok 총 데이터 개수: 300
y_fault 총 데이터 개수: 1200
train_data_80_ok 학습 데이터 개수: 192
train_data_80_fault 학습 데이터 개수: 768
test_data_ok 테스트 데이터 개수: 60
test_data_fault 테스트 데이터 개수: 240
valid_data_ok 검증 데이터 개수: 48
valid_data_fault 검증 데이터 개수: 192


# 테스트 데이터

In [8]:
# # 이미지 복사
# import shutil

# # 폴더 초기화 함수: 기존 폴더 삭제 후 새로 생성
# def prepare_output_dir(output_dir, data):
#     if os.path.exists(output_dir):
#         shutil.rmtree(output_dir)  # 폴더와 내부 파일 모두 삭제
#     os.makedirs(output_dir, exist_ok=True)  # 새로 생성
#     copy_images(data, output_dir)


# def copy_images(data, output_dir):
#     image_dir = os.path.join(output_dir, 'images')
#     os.makedirs(image_dir, exist_ok=True)

#     for index, row in data.iterrows():
#         image_path = row['path']
#         shutil.copy(image_path, image_dir)


# # 학습 데이터와 검증 데이터를 각각 처리
# prepare_output_dir('./train', train_data_80_ok)
# prepare_output_dir('./test', test_data_ok)
# prepare_output_dir('./valid', valid_data_ok)
# prepare_output_dir('./train', train_data_80_fault)
# prepare_output_dir('./test', test_data_fault)
# prepare_output_dir('./valid', valid_data_fault)


### bboxes 데이터 추출

In [9]:
import ast

def parse_bboxes(data):
    """
    문자열 형태의 bounding box 정보를 파싱하여 리스트 형태로 변환하는 함수

    Args:
        data: 파싱할 데이터셋

    Returns:
        list: 파싱된 bounding box 리스트
    """

    bboxes_list = []
    for bbox_str in data["bboxes"]:
        try:
            bboxes = ast.literal_eval(bbox_str)
            bbox_list = []
            for bbox in bboxes:
                x, y, w, h = bbox
                bbox_list.append([x, y, w, h])
            bboxes_list.append(bbox_list)
        except (ValueError, SyntaxError) as e:
            print(f"Error parsing bbox: {e}")
    return bboxes_list

# 데이터셋별 bounding box 파싱
train_ok_bboxes = parse_bboxes(train_data_80_ok)
valid_ok_bboxes = parse_bboxes(valid_data_ok)
test_ok_bboxes = parse_bboxes(test_data_ok)
train_fault_bboxes = parse_bboxes(train_data_80_fault)
valid_fault_bboxes = parse_bboxes(valid_data_fault)
test_fault_bboxes = parse_bboxes(test_data_fault)

# 결과 출력 (예시)
# print(train_ok_bboxes[:5])
# print(valid_ok_bboxes[:5])
# print(test_ok_bboxes[:5])
# print(train_fault_bboxes[:5])
# print(valid_fault_bboxes[:5])
# print(test_fault_bboxes[:5])

# .yaml label 데이터 생성

### 1. 디렉토리 경로 전처리

In [10]:
import os
from PIL import Image


def extract_image_info(image_paths):
    """
    이미지 경로 리스트에서 이미지 크기, 디렉토리 경로, 파일 이름을 추출하는 함수

    Args:
        image_paths (list): 이미지 경로 리스트

    Returns:
        tuple: (image_sizes, image_dirs, file_names)
            image_sizes (list): 이미지 크기 리스트
            image_dirs (list): 이미지 디렉토리 경로 리스트
            file_names (list): 이미지 파일 이름 리스트
    """

    image_sizes = []
    image_dirs = []
    file_names = []

    for path in image_paths:
        try:
            image = Image.open(path)
            width, height = image.size
            image_sizes.append((width, height))
            image_dirs.append(os.path.dirname(path) + os.sep)
            file_names.append(os.path.splitext(os.path.basename(path))[0])
        except FileNotFoundError:
            print(f"파일을 찾을 수 없습니다: {path}")

    return image_sizes, image_dirs, file_names


# 데이터 준비 (예시)
train_ok_image_paths = train_data_80_ok.iloc[:, 0].tolist()
valid_ok_image_paths = valid_data_ok.iloc[:, 0].tolist()
test_ok_image_paths = test_data_ok.iloc[:, 0].tolist()
train_fault_image_paths = train_data_80_fault.iloc[:, 0].tolist()
valid_fault_image_paths = valid_data_fault.iloc[:, 0].tolist()
test_fault_image_paths = test_data_fault.iloc[:, 0].tolist()

# 이미지 정보 추출
train_sizes_ok, train_dirs_ok, train_names_ok = extract_image_info(train_ok_image_paths)
valid_sizes_ok, valid_dirs_ok, valid_names_ok = extract_image_info(valid_ok_image_paths)
test_sizes_ok, test_dirs_ok, test_names_ok = extract_image_info(test_ok_image_paths)
train_sizes_fault, train_dirs_fault, train_names_fault = extract_image_info(train_fault_image_paths)
valid_sizes_fault, valid_dirs_fault, valid_names_fault = extract_image_info(valid_fault_image_paths)
test_sizes_fault, test_dirs_fault, test_names_fault = extract_image_info(test_fault_image_paths)

# 결과 출력 ok
# print("train_sizes_ok 이미지 크기:", train_sizes_ok[:5])
# print("train_dirs_ok 이미지 디렉토리:", train_dirs_ok[:5])
# print("train_names_ok 파일 이름:", train_names_ok[:5])
# print()
# print("valid_sizes_ok 이미지 크기:", valid_sizes_ok[:5])
# print("valid_dirs_ok 이미지 디렉토리:", valid_dirs_ok[:5])
# print("valid_names_ok 파일 이름:", valid_names_ok[:5])
# print()
# print("test_sizes_ok 이미지 크기:", test_sizes_ok[:5])
# print("test_dirs_ok 이미지 디렉토리:", test_dirs_ok[:5])
# print("test_names_ok 파일 이름:", test_names_ok[:5])
# 결과 출력 fault
# print("train_sizes_fault 이미지 크기:", train_sizes_fault[:5])
# print("train_dirs_fault 이미지 디렉토리:", train_dirs_fault[:5])
# print("train_names_fault 파일 이름:", train_names_fault[:5])
# print()
# print("valid_sizes_fault 이미지 크기:", valid_sizes_fault[:5])
# print("valid_dirs_fault 이미지 디렉토리:", valid_dirs_fault[:5])
# print("valid_names_fault 파일 이름:", valid_names_fault[:5])
# print()
# print("test_sizes_fault 이미지 크기:", test_sizes_fault[:5])
# print("test_dirs_fault 이미지 디렉토리:", test_dirs_fault[:5])
# print("test_names_fault 파일 이름:", test_names_fault[:5])

c:\Users\dawoo\Desktop\SF7\coding\PYTORCH\pytorch-env\Lib\site-packages\PIL\Image.py:3186: DecompressionBombWarning: Image size (108000000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


### 3. 부품 상태 전처리

In [11]:
# 부품 상태를 체크하는 함수
def get_label(part, status):
    if part == "범퍼":
        if status == 1:
            return 0
        elif status == 0:
            return 1
    elif part == "도어":
        if status == 1:
            return 0
        elif status == 0:
            return 2
    elif part == "휀더":
        if status == 1:
            return 0
        elif status == 0:
            return 3
    elif part == "프레임":
        if status == 1:
            return 0
        elif status == 0:
            return 4
    return None  # 기본적으로 반환할 값이 없으면 None

# 예시 부품과 상태
parts_and_status_train_ok = []
parts_and_status_valid_ok = []
parts_and_status_test_ok = []
for quality, part in zip(train_data_80_ok.iloc[:, 1], train_data_80_ok.iloc[:, 2]):
    parts_and_status_train_ok.append({"part": part, "status": quality})

for quality, part in zip(test_data_ok.iloc[:, 1], test_data_ok.iloc[:, 2]):
    parts_and_status_test_ok.append({"part": part, "status": quality})

for quality, part in zip(valid_data_ok.iloc[:, 1], valid_data_ok.iloc[:, 2]):
    parts_and_status_valid_ok.append({"part": part, "status": quality})

parts_and_status_train_fault = []
parts_and_status_valid_fault = []
parts_and_status_test_fault = []
for quality, part in zip(train_data_80_fault.iloc[:, 1], train_data_80_fault.iloc[:, 2]):
    parts_and_status_train_fault.append({"part": part, "status": quality})

for quality, part in zip(test_data_fault.iloc[:, 1], test_data_fault.iloc[:, 2]):
    parts_and_status_test_fault.append({"part": part, "status": quality})

for quality, part in zip(valid_data_fault.iloc[:, 1], valid_data_fault.iloc[:, 2]):
    parts_and_status_valid_fault.append({"part": part, "status": quality})

# print(f"parts_and_status_train : {parts_and_status_train_ok}")
# print(f"parts_and_status_test : {parts_and_status_test_ok}")
# print(f"parts_and_status_valid : {parts_and_status_valid_ok}")

# print(f"parts_and_status_train : {parts_and_status_train_fault}")
# print(f"parts_and_status_test : {parts_and_status_test_fault}")
# print(f"parts_and_status_valid : {parts_and_status_valid_fault}")

# 라벨링 결과를 저장할 리스트
labels_train_ok = []
labels_valid_ok = []
label_test_ok = []

# 각 부품 상태에 대해 라벨을 설정
for item in parts_and_status_train_ok:
    label = get_label(item["part"], item["status"])
    labels_train_ok.append(label)
for item in parts_and_status_test_ok:
    label = get_label(item["part"], item["status"])
    labels_valid_ok.append(label)
for item in parts_and_status_valid_ok:
    label = get_label(item["part"], item["status"])
    label_test_ok.append(label)

# # 결과 출력
# print("labels_train_ok 라벨링 결과 :", labels_train_ok)
# print("labels_valid_ok 라벨링 결과 :", labels_valid_ok)
# print("label_test_ok 라벨링 결과 :", label_test_ok)

# 라벨링 결과를 저장할 리스트
labels_train_fault = []
labels_valid_fault = []
label_test_fault = []

# 각 부품 상태에 대해 라벨을 설정
for item in parts_and_status_train_fault:
    label = get_label(item["part"], item["status"])
    labels_train_fault.append(label)
for item in parts_and_status_test_fault:
    label = get_label(item["part"], item["status"])
    labels_valid_fault.append(label)
for item in parts_and_status_valid_fault:
    label = get_label(item["part"], item["status"])
    label_test_fault.append(label)

# # 결과 출력
# print("labels_train_fault 라벨링 결과 :", labels_train_fault)
# print("labels_valid_fault 라벨링 결과 :", labels_valid_fault)
# print("label_test_fualt 라벨링 결과 :", label_test_fault)

In [12]:
import os
import shutil

# 폴더 초기화 함수
def prepare_output_dir(output_dir):
    if os.path.exists(output_dir):
        shutil.rmtree(output_dir)  # 기존 폴더 삭제
    os.makedirs(output_dir, exist_ok=True)  # 새 폴더 생성

# 이미지 복사 함수
def copy_images(output_dir, data):
    image_dir = os.path.join(output_dir, 'images')
    
    # 이미지 폴더가 존재하는지 확인하고, 없으면 새로 생성
    if not os.path.exists(image_dir):
        os.makedirs(image_dir)

    for index, row in data.iterrows():
        image_path = row['path']
        if os.path.exists(image_path):  # 이미지 경로 확인
            shutil.copy(image_path, image_dir)
        else:
            print(f"[WARNING] Image not found: {image_path}")

# YOLO 형식 변환 함수
def convert_to_yolo_format(bbox, label, image_width, image_height):
    yolo_labels = []

    if not bbox or len(bbox) == 0:  # 바운딩 박스가 없을 경우
        yolo_labels.append(f"{label} 0.0 0.0 0.0 0.0")
    else:
        for x_min, y_min, width, height in bbox:
            x_center = (x_min + width / 2) / image_width
            y_center = (y_min + height / 2) / image_height
            norm_width = width / image_width
            norm_height = height / image_height
            yolo_labels.append(f"{label} {x_center:.6f} {y_center:.6f} {norm_width:.6f} {norm_height:.6f}")

    return yolo_labels

# YOLO 라벨 저장 함수
def save_yolo_labels(output_dir, bboxes, labels, size, names):
    label_dir = os.path.join(output_dir, 'labels')

    # 라벨 폴더가 존재하는지 확인하고, 없으면 새로 생성
    if not os.path.exists(label_dir):
        os.makedirs(label_dir)

    for bbox, label, (w, h), name in zip(bboxes, labels, size, names):
        yolo_labels = convert_to_yolo_format(bbox, label, w, h)

        txt_file_path = os.path.join(label_dir, f"{name}.txt")
        with open(txt_file_path, 'w') as txt_file:
            for yolo_data in yolo_labels:
                txt_file.write(f"{yolo_data}\n")

        # 디버깅 로그 추가
        print(f"[INFO] Saved YOLO label: {txt_file_path} (bbox count: {len(bbox)})")

# 이미지 복사 및 라벨 저장 작업을 한 번만 처리하도록 수정
def process_data(output_dir, data, bboxes, labels, sizes, names):
    
    copy_images(output_dir, data)  # 이미지 복사
    save_yolo_labels(output_dir, bboxes, labels, sizes, names)  # YOLO 라벨 저장

# 디렉토리 설정
train_output_dir = '.\\train'
valid_output_dir = '.\\valid'
test_output_dir = '.\\test'
prepare_output_dir(train_output_dir)
prepare_output_dir(valid_output_dir)
prepare_output_dir(test_output_dir)

# 이미지 및 라벨 처리
process_data(train_output_dir, train_data_80_ok, train_ok_bboxes, labels_train_ok, train_sizes_ok, train_names_ok)
process_data(test_output_dir, test_data_ok, test_ok_bboxes, label_test_ok, test_sizes_ok, test_names_ok)
process_data(valid_output_dir, valid_data_ok, valid_ok_bboxes, labels_valid_ok, valid_sizes_ok, valid_names_ok)

process_data(train_output_dir, train_data_80_fault, train_fault_bboxes, labels_train_fault, train_sizes_fault, train_names_fault)
process_data(test_output_dir, test_data_fault, test_fault_bboxes, label_test_fault, test_sizes_fault, test_names_fault)
process_data(valid_output_dir, valid_data_fault, valid_fault_bboxes, labels_valid_fault, valid_sizes_fault, valid_names_fault)


[INFO] Saved YOLO label: .\train\labels\205_101_10_1419f342-b811-454f-a371-57c02c6343c8.txt (bbox count: 0)
[INFO] Saved YOLO label: .\train\labels\205_101_10_60da5008-10b2-40b9-a0f6-e7dc22d2f11c.txt (bbox count: 0)
[INFO] Saved YOLO label: .\train\labels\209_102_10_319c5ffd-6e52-4711-a74f-bee9f0431a4f.txt (bbox count: 0)
[INFO] Saved YOLO label: .\train\labels\207_204_10_fa45f552-bbf7-47b6-b40c-a6cf525c7d2e.txt (bbox count: 0)
[INFO] Saved YOLO label: .\train\labels\204_101_10_656af499-f95d-4d32-b8b4-542814a9ce18.txt (bbox count: 0)
[INFO] Saved YOLO label: .\train\labels\205_101_10_3e1ba5e1-eaa5-4dcb-a088-67667d32c221.txt (bbox count: 0)
[INFO] Saved YOLO label: .\train\labels\204_101_10_395c646b-88fe-433f-ace8-73cd04ac7fa9.txt (bbox count: 0)
[INFO] Saved YOLO label: .\train\labels\209_102_10_ec22ac26-687a-4b85-b622-db2ed6b5b45a.txt (bbox count: 0)
[INFO] Saved YOLO label: .\train\labels\205_101_10_26291af5-21d5-4e8c-9b03-f5bb04e31da0.txt (bbox count: 0)
[INFO] Saved YOLO label: .\t

In [13]:
# import yaml

# # YAML 파일에 저장
# yaml_data = {
#     "train": "C:/Users/dawoo/desktop/sf7/coding/pytorch/project/train/images",
#     "test": "C:/Users/dawoo/desktop/sf7/coding/pytorch/project/test/images",
#     "val": "C:/Users/dawoo/desktop/sf7/coding/pytorch/project/valid/images",
#     "nc": 5,
#     "names": ["ok",
#               "bumper_faulty",
#               "door_faulty",
#               "fender_faulty",
#               "frame_faulty"],
#     "augment": {
#         "flipud": 0.5,  # 상하 반전 확률
#         "fliplr": 0.5,  # 좌우 반전 확률
#         "hsv_h": 0.015,  # Hue 변화 범위
#         "hsv_s": 0.7,   # 채도 변화 범위
#         "hsv_v": 0.4,   # 밝기 변화 범위
#         "translate": 0.1,  # 번역 범위
#         "scale": 0.5,  # 크기 조정 범위
#         "rotate": 30,   # 회전 범위 (degrees)
#         "mosaic": 1.0,  # Mosaic 증강 비율
#         "mixup": 0.2,  # MixUp 증강 비율
#         "cutmix": 0.2,  # CutMix 증강 비율

#         # 추가 증강 옵션
#         "motion_blur": 0.3,  # 모션 블러 추가 (확률)
#         "gauss_noise": 0.3,  # 가우시안 노이즈 추가 (확률)
#         "random_shadow": 0.3,  # 랜덤 그림자 추가 (확률)
#         "random_fog": 0.2,  # 랜덤 안개 효과 추가 (확률)
#         "perspective": 0.2,  # 원근 변환 (확률)
#         "shear": 15,  # 전단 변환 (degree)
#         "crop": 0.3,  # 랜덤 크롭 (확률)
#         "brightness": 0.2,  # 밝기 조정 (확률)
#         "contrast": 0.2,  # 대비 조정 (확률)
#         "sharpen": 0.3  # 샤프닝 효과 (확률)
#     }
# }

# with open("data.yaml", "w") as yaml_file:
#     yaml.dump(yaml_data, yaml_file, default_flow_style=False, sort_keys=False)

# print("YAML 파일이 생성되었습니다.")


## 모델 코드 작성

In [14]:
# from ultralytics import YOLO

# # YOLO 모델 객체 생성
# model = YOLO("yolov8s.pt")  # 사전 학습된 YOLOv8 모델 로드

# # 모델 학습 시작
# results = model.train(
#     data="data.yaml",       # YAML 파일 경로
#     epochs=50,              # 학습 에포크 수
#     imgsz=640,              # 입력 이미지 크기 (기본값: 640)
#     batch=16,               # 배치 크기
#     project="C:\\Users\\dawoo\\Desktop\\SF7\\coding\\PYTORCH\\project\\YOLO", # 학습 결과를 저장할 프로젝트 이름
#     name="part_damage_detection",   # 저장될 실행 이름
#     workers=4,              # 데이터 로드 병렬 처리 수
#     device=0                # GPU ID (0은 첫 번째 GPU를 의미, -1은 CPU)
# )

# model.save('part_damage_detection.pt')

In [15]:
# import os
# from ultralytics import YOLO

# # 데이터 설정 파일 경로
# data_yaml = "data.yaml"

# # YOLO 모델 정의 (랜덤 초기화로 학습 시작)
# model = YOLO("yolov8s.pt")

# # 학습 설정
# model.train(
#     data=data_yaml,          # 데이터 설정 파일 경로
#     epochs=60,               # 학습 에폭 수
#     batch=16,                # 배치 크기
#     imgsz=640,               # 입력 이미지 크기
#     device=0,                # GPU 사용
#     workers=4,               # 데이터 로드 병렬 처리 워커 수
#     project="C:\\Users\\dawoo\\Desktop\\SF7\\coding\\PYTORCH\\project\\YOLO",     # 결과 저장 디렉토리
#     name="part_damage_detection",       # 실험 이름
#     conf=0.35,
#     lr0=0.005,  # 초기 학습률
#     lrf=0.1,                # 학습률 감소율 설정
#     optimizer='Adamw',  # 옵티마이저 설정 (SGD, Adam, AdamW 등)
#     weight_decay=0.001,   # 가중치 감소 설정
#     patience=20,
# )
# model.save('model_8_s_60_16_960.pt')

In [16]:
# import os
# from ultralytics import YOLO

# # 데이터 설정 파일 경로
# data_yaml = "data.yaml"

# # YOLO 모델 정의 (Medium 모델로 변경)
# model = YOLO("yolov8m.pt")

# # 학습 설정
# model.train(
#     data=data_yaml,  # 데이터 설정 파일 경로
#     epochs=80,  # 학습 에폭 수 조정
#     batch=8,  # 배치 크기를 줄여 안정적인 학습 유도
#     imgsz=640,  # 입력 이미지 크기
#     device=0,  # GPU 사용
#     workers=4,  # 데이터 로드 병렬 처리 워커 수
#     project="C:\\Users\\dawoo\\Desktop\\SF7\\coding\\PYTORCH\\project\\YOLO",
#     name="part_damage_detection_v2",
#     conf=0.35,  # Confidence threshold
#     lr0=0.003,  # 낮은 초기 학습률
#     lrf=0.2,  # 학습률 감소율 증가
#     optimizer='Adamw',  # 옵티마이저 설정
#     weight_decay=0.0005,  # 가중치 감소 설정
#     patience=15,  # Early stopping 설정
#     warmup_epochs=5,  # Warmup 에폭 증가
#     cos_lr=True,  # 코사인 스케줄링 사용
#     augment=True,  # 데이터 증강 활성화
#     hsv_h=0.015,  # Hue variation
#     hsv_s=0.7,  # Saturation variation
#     hsv_v=0.4,  # Value variation
#     flipud=0.5,  # 수직 뒤집기 확률
#     fliplr=0.5,  # 수평 뒤집기 확률
#     mosaic=0.5,  # 모자이크 증강 비율 감소
#     mixup=0.2,  # Mixup 증강
# )
# # 학습 완료 후 모델 저장
# model.save('model_8_m_80_8_640.pt')


In [17]:
# import os
# from ultralytics import YOLO

# # 데이터 설정 파일 경로
# data_yaml = "data.yaml"

# # YOLO 모델 정의 (Medium 모델 사용)
# model = YOLO("yolov8s.pt")

# # 학습 설정
# model.train(
#     data=data_yaml,  # 데이터 설정 파일 경로
#     epochs=50,  # 학습 에폭 수
#     batch=16,  # 배치 크기 조정 (메모리 초과 방지)
#     imgsz=640,  # 입력 이미지 크기
#     device=0,  # GPU 사용
#     workers=4,  # 데이터 로드 병렬 처리 워커 수
#     project="C:\\Users\\dawoo\\Desktop\\SF7\\coding\\PYTORCH\\project\\YOLO",
#     name="part_damage_detection_v2",
#     conf=0.4,  # Confidence threshold
#     optimizer='Adamw',  # 옵티마이저 설정
#     weight_decay=0.0005,  # 가중치 감소 설정
#     patience=15,  # Early stopping 여유 증대
#     cos_lr=True,  # 코사인 스케줄링 활성화
#     augment=True,  # 기본 데이터 증강 활성화
#     # 데이터 증강 추가
#     hsv_h=0.015,  # Hue variation
#     hsv_s=0.7,  # Saturation variation
#     hsv_v=0.4,  # Value variation
#     flipud=0.5,  # 수직 뒤집기 확률
#     fliplr=0.5,  # 수평 뒤집기 확률
#     mosaic=0.5,  # 모자이크 증강 비율 감소
#     mixup=0.2,  # Mixup 증강
#     cache=True,  # 데이터 로딩 속도를 높이기 위해 데이터 캐싱
# )
# # 학습 완료 후 모델 저장
# model.save('model_8_s_50_16_960.pt')


In [18]:
# import cv2
# import os
# from ultralytics import YOLO
# import numpy as np

# # 이미지 전처리 함수 (노이즈 제거)


# def denoise_image(image):
#     # 이미지에서 Gaussian Blur 적용 (노이즈 제거)
#     return cv2.GaussianBlur(image, (5, 5), 0)



# # 데이터셋 경로
# train_images_dir = "./train/images"
# vaild_images_dir = "./valid/images"

# def preprocess_images(image_dir):
#     image_paths = [os.path.join(image_dir, f) for f in os.listdir(
#         image_dir) if f.endswith('.jpg') or f.endswith('.png') or f.endswith(".jpeg")]
#     for image_path in image_paths:
#         image = cv2.imread(image_path)
#         denoised_image = denoise_image(image)
#         cv2.imwrite(image_path, denoised_image)  # 덮어쓰기 (또는 다른 디렉토리에 저장)


# # 이미지 전처리 실행
# preprocess_images(train_images_dir)
# preprocess_images(vaild_images_dir)

In [19]:
# import cv2
# import os
# from ultralytics import YOLO
# import numpy as np

# # 학습할 이미지에 노이즈 제거 적용


# # 데이터 설정 파일 경로
# data_yaml = "data.yaml"

# # YOLO 모델 정의 (Small 모델 사용)
# model = YOLO("yolov8s.pt")


# # 학습 설정
# model.train(
#     data=data_yaml,  # 데이터 설정 파일 경로
#     epochs=50,  # 학습 에폭 수
#     batch=16,  # 배치 크기 조정 (메모리 초과 방지)
#     imgsz=640,  # 입력 이미지 크기
#     device=0,  # GPU 사용
#     workers=4,  # 데이터 로드 병렬 처리 워커 수
#     project="C:\\Users\\dawoo\\Desktop\\SF7\\coding\\PYTORCH\\project\\YOLO",
#     name="part_damage_detection_v2",
#     conf=0.4,  # Confidence threshold
#     optimizer='Adamw',  # 옵티마이저 설정
#     weight_decay=0.0005,  # 가중치 감소 설정
#     patience=15,  # Early stopping 여유 증대
#     cos_lr=True,  # 코사인 스케줄링 활성화
#     augment=True,  # 기본 데이터 증강 활성화
#     # 데이터 증강 추가
#     hsv_h=0.015,  # Hue variation
#     hsv_s=0.7,  # Saturation variation
#     hsv_v=0.4,  # Value variation
#     flipud=0.5,  # 수직 뒤집기 확률
#     fliplr=0.5,  # 수평 뒤집기 확률
#     mosaic=0.5,  # 모자이크 증강 비율 감소
#     mixup=0.2,  # Mixup 증강
#     cache=True,  # 데이터 로딩 속도를 높이기 위해 데이터 캐싱
# )

# # 학습 완료 후 모델 저장
# model.save('model_8_s_50_16_960.pt')

In [20]:
# import os
# from ultralytics import YOLO

# # 데이터 설정 파일 경로
# data_yaml = "data.yaml"

# # YOLO 모델 정의 (Medium 모델로 변경)
# model = YOLO("yolov8m.pt")

# # 학습 설정
# model.train(
#     data=data_yaml,  # 데이터 설정 파일 경로
#     epochs=10,  # 학습 에폭 수 조정
#     batch=8,  # 배치 크기를 줄여 안정적인 학습 유도
#     imgsz=640,  # 입력 이미지 크기
#     device=0,  # GPU 사용
#     workers=4,  # 데이터 로드 병렬 처리 워커 수
#     project="C:\\Users\\dawoo\\Desktop\\SF7\\coding\\PYTORCH\\project\\YOLO",
#     name="part_damage_detection_v2",
#     conf=0.4,  # Confidence threshold
#     optimizer='Adamw',  # 옵티마이저 설정
#     weight_decay=0.0005,  # 가중치 감소 설정
#     augment=True,  # 데이터 증강 활성화
#     hsv_h=0.015,  # Hue variation
#     hsv_s=0.7,  # Saturation variation
#     hsv_v=0.4,  # Value variation
#     flipud=0.5,  # 수직 뒤집기 확률
#     fliplr=0.5,  # 수평 뒤집기 확률
#     mosaic=0.5,  # 모자이크 증강 비율 감소
#     mixup=0.2,  # Mixup 증강
#     cache=True,  # 데이터 로딩 속도를 높이기 위해 데이터 캐싱
# )
# # 학습 완료 후 모델 저장
# model.save('model_8_m_10_8_640.pt')


In [21]:
# import os
# from ultralytics import YOLO

# # 데이터 설정 파일 경로
# data_yaml = "data.yaml"

# # YOLO 모델 정의 (yolov8m.pt 사용)
# model = YOLO("yolov8n.pt")

# # 학습 설정
# model.train(
#     data=data_yaml,
#     epochs=70,
#     batch=8,  # 배치 크기 감소
#     imgsz=640,  # 이미지 크기
#     device=0,
#     workers=3,
#     project="YOLO1",
#     name="part_damage_detection",
#     conf=0.5,
#     lr0=0.0001,
#     optimizer='AdamW',
#     weight_decay=0.01,
#     patience=10,
# )

# model.save('part_damage_detection2.pt')

In [22]:
# import os
# from ultralytics import YOLO

# # 데이터 설정 파일 경로
# data_yaml = "data.yaml"

# # YOLO 모델 정의 (yolov8m.pt 사용)
# model = YOLO("yolov8s.pt")

# # 학습 설정
# model.train(
#     data=data_yaml,
#     epochs=50,
#     batch=8,  # 배치 크기 감소
#     imgsz=640,  # 이미지 크기 줄이기
#     device=0,
#     workers=4,
#     project="YOLO2",
#     name="part_damage_detection",
#     conf=0.5,
#     lr0=0.0005,
#     optimizer='AdamW',
#     weight_decay=0.005,
#     patience=10,
# )

# model.save('part_damage_detection3.pt')

In [23]:
# import cv2
# import os
# from ultralytics import YOLO
# import numpy as np

# # 학습할 이미지에 노이즈 제거 적용


# # 데이터 설정 파일 경로
# data_yaml = "data.yaml"

# # YOLO 모델 정의 (Small 모델 사용)
# model = YOLO("yolov8s.pt")


# # 학습 설정
# model.train(
#     data=data_yaml,  # 데이터 설정 파일 경로
#     epochs=80,  # 학습 에폭 수
#     batch=16,  # 배치 크기 조정 (메모리 초과 방지)
#     imgsz=640,  # 입력 이미지 크기
#     device=0,  # GPU 사용
#     workers=4,  # 데이터 로드 병렬 처리 워커 수
#     project="C:\\Users\\dawoo\\Desktop\\SF7\\coding\\PYTORCH\\project\\YOLO",
#     name="part_damage_detection_v2",
#     conf=0.3,  # Confidence threshold
#     optimizer='Adamw',  # 옵티마이저 설정
#     weight_decay=0.0005,  # 가중치 감소 설정
#     cos_lr=True,  # 코사인 스케줄링 활성화
#     augment=True,  # 기본 데이터 증강 활성화
#     cache=True,  # 데이터 로딩 속도를 높이기 위해 데이터 캐싱
# )

# # 학습 완료 후 모델 저장
# model.save('model_8_s_80_16_960.pt')

In [24]:
# import yaml
# from ultralytics import YOLO

# # YAML 파일에 저장
# yaml_data = {
#     "train": "C:/Users/dawoo/desktop/sf7/coding/pytorch/project/train/images",
#     "test": "C:/Users/dawoo/desktop/sf7/coding/pytorch/project/test/images",
#     "val": "C:/Users/dawoo/desktop/sf7/coding/pytorch/project/valid/images",
#     "nc": 5,
#     "names": ["ok", "bumper_faulty", "door_faulty", "fender_faulty", "frame_faulty"],
#     "augment": {
#         "flipud": 0.5,
#         "fliplr": 0.5,
#         "hsv_h": 0.015,
#         "hsv_s": 0.7,
#         "hsv_v": 0.4,
#         "translate": 0.1,
#         "scale": 0.5,
#         "rotate": 15,  # 회전 범위 수정
#         "mosaic": 0.8,  # 모자이크 증강 비율 수정
#         "mixup": 0.2,
#         "cutmix": 0.1,
#         "motion_blur": 0.2,
#         "gauss_noise": 0.2,
#         "random_shadow": 0.2,
#         "random_fog": 0.1,
#         "shear": 10,  # 전단 변환 (degree)
#         "crop": 0.2,
#         "brightness": 0.2,
#         "contrast": 0.2,
#         "sharpen": 0.2
#     }
# }

# with open("data.yaml", "w") as yaml_file:
#     yaml.dump(yaml_data, yaml_file, default_flow_style=False, sort_keys=False)
#     print("YAML 파일이 생성되었습니다.")

In [25]:
# # 모델 로드
# model = YOLO("yolov8s.pt")

# # 학습 설정
# model.train(
#     data="data.yaml",  # 데이터 설정 파일 경로
#     epochs=80,  # 학습 에폭 수
#     batch=16,  # 배치 크기
#     imgsz=640,  # 입력 이미지 크기
#     device=0,  # GPU 사용
#     workers=4,  # 데이터 로드 병렬 처리 워커 수
#     project="C:\\Users\\dawoo\\Desktop\\SF7\\coding\\PYTORCH\\project\\YOLO",
#     name="part_damage_detection_v2",
#     conf=0.3,  # Confidence threshold
#     optimizer='Adamw',  # AdamW 옵티마이저
#     weight_decay=0.0005,  # 가중치 감소 설정
#     cos_lr=True,  # 코사인 스케줄링 활성화
#     augment=True,  # 기본 데이터 증강 활성화
#     cache=True,  # 데이터 로딩 속도 개선
#     patience=25,  # Early Stopping 여유 증대
#     lr0=0.001,  # 초기 학습률 설정
#     lrf=0.1,  # 학습률 스케줄링 비율
#     momentum=0.937,  # 모멘텀 설정
# )

# # 학습 완료 후 모델 저장
# model.save('model_8_s_80_16_960.pt')

In [26]:
# import yaml
# from ultralytics import YOLO

# # YAML 파일에 저장
# yaml_data = {
#     "train": "C:/Users/dawoo/desktop/sf7/coding/pytorch/project/train/images",
#     "test": "C:/Users/dawoo/desktop/sf7/coding/pytorch/project/test/images",
#     "val": "C:/Users/dawoo/desktop/sf7/coding/pytorch/project/valid/images",
#     "nc": 5,
#     "names": ["ok", "bumper_faulty", "door_faulty", "fender_faulty", "frame_faulty"],
#     "augment": {
#         "flipud": 0.5,
#         "fliplr": 0.5,
#         "hsv_h": 0.015,
#         "hsv_s": 0.7,
#         "hsv_v": 0.4,
#         "translate": 0.1,
#         "scale": 0.5,
#         "rotate": 10,  # 회전 범위를 조금 줄임
#         "mosaic": 0.7,  # 모자이크 증강 비율을 줄여서 안정성 강화
#         "mixup": 0.2,
#         "cutmix": 0.1,
#         "motion_blur": 0.2,
#         "gauss_noise": 0.2,
#         "random_shadow": 0.2,
#         "random_fog": 0.1,
#         "shear": 10,
#         "crop": 0.2,
#         "brightness": 0.2,
#         "contrast": 0.2,
#         "sharpen": 0.2
#     }
# }

# with open("data.yaml", "w") as yaml_file:
#     yaml.dump(yaml_data, yaml_file, default_flow_style=False, sort_keys=False)
#     print("YAML 파일이 생성되었습니다.")

In [27]:
# # 모델 로드
# model = YOLO("yolov8s.pt")

# # 학습 설정
# model.train(
#     data="data.yaml",  # 데이터 설정 파일 경로
#     epochs=100,  # 에폭을 100으로 늘려서 더 충분히 학습
#     batch=16,  # 배치 크기
#     imgsz=640,  # 입력 이미지 크기
#     device=0,  # GPU 사용
#     workers=4,  # 데이터 로드 병렬 처리 워커 수
#     project="C:\\Users\\dawoo\\Desktop\\SF7\\coding\\PYTORCH\\project\\YOLO",
#     name="part_damage_detection_v2",
#     conf=0.3,  # Confidence threshold
#     optimizer='AdamW',  # AdamW 옵티마이저 사용
#     weight_decay=0.0005,  # 가중치 감소 설정
#     cos_lr=True,  # 코사인 스케줄링 활성화
#     augment=True,  # 기본 데이터 증강 활성화
#     cache=True,  # 데이터 로딩 속도 개선
#     patience=25,  # Early Stopping 여유 증대
#     lr0=0.0005,  # 초기 학습률을 낮춤
#     lrf=0.1,  # 학습률 스케줄링 비율
#     momentum=0.85,  # 모멘텀 설정
# )

# # 학습 완료 후 모델 저장
# model.save('model_8_s_80_16_960.pt')

In [28]:
# import yaml
# from ultralytics import YOLO

# # YAML 파일에 저장
# yaml_data = {
#     "train": "C:/Users/dawoo/desktop/sf7/coding/pytorch/project/train/images",
#     "test": "C:/Users/dawoo/desktop/sf7/coding/pytorch/project/test/images",
#     "val": "C:/Users/dawoo/desktop/sf7/coding/pytorch/project/valid/images",
#     "nc": 5,
#     "names": ["ok", "bumper_faulty", "door_faulty", "fender_faulty", "frame_faulty"],
#     "augment": {
#         "flipud": 0.5,
#         "fliplr": 0.5,
#         "hsv_h": 0.015,
#         "hsv_s": 0.7,
#         "hsv_v": 0.4,
#         "translate": 0.1,
#         "scale": 0.5,
#         "rotate": 15,  # 회전 범위를 늘려서 다양한 방향에 대해 학습
#         "mosaic": 0.8,  # 모자이크 증강 비율을 높여서 더 많은 변형 데이터 생성
#         "mixup": 0.3,  # mixup 비율을 증가시켜 다양한 이미지 결합
#         "cutmix": 0.15,  # cutmix 비율을 살짝 증가
#         "motion_blur": 0.25,  # motion blur를 강하게 적용
#         "gauss_noise": 0.25,  # 노이즈를 더 많이 추가
#         "random_shadow": 0.3,  # 그림자 효과를 증가시켜 밝기 변화에 강하게
#         "random_fog": 0.2,  # 안개 효과를 늘려서 다양한 시나리오 학습
#         "shear": 20,  # 전단 변환 정도를 더 강하게
#         "crop": 0.3,  # 잘라내기 영역을 증가시켜 모델의 일반화 성능 향상
#         "brightness": 0.3,  # 밝기 변화 범위 증가
#         "contrast": 0.3,  # 대비 범위 증가
#         "sharpen": 0.3  # 이미지 샤프닝 강도를 높임
#     }
# }

# with open("data.yaml", "w") as yaml_file:
#     yaml.dump(yaml_data, yaml_file, default_flow_style=False, sort_keys=False)
#     print("YAML 파일이 생성되었습니다.")


In [29]:
# # 모델 로드
# model = YOLO("yolov8s.pt")

# # 학습 설정
# model.train(
#     data="data.yaml",  # 데이터 설정 파일 경로
#     epochs=100,  # 에폭 수를 늘려서 더 많은 학습 진행
#     batch=16,  # 배치 크기
#     imgsz=640,  # 입력 이미지 크기
#     device=0,  # GPU 사용
#     workers=4,  # 데이터 로드 병렬 처리 워커 수
#     project="C:\\Users\\dawoo\\Desktop\\SF7\\coding\\PYTORCH\\project\\YOLO",
#     name="part_damage_detection_v2",
#     conf=0.3,  # Confidence threshold
#     optimizer='Adamw',  # AdamW 옵티마이저
#     weight_decay=0.0005,  # 가중치 감소 설정
#     cos_lr=True,  # 코사인 스케줄링 활성화
#     augment=True,  # 기본 데이터 증강 활성화
#     cache=True,  # 데이터 로딩 속도 개선
#     patience=25,  # Early Stopping 여유 증대
#     lr0=0.001,  # 초기 학습률 설정
#     lrf=0.1,  # 학습률 스케줄링 비율
#     momentum=0.85,  # 모멘텀을 증가시켜 더 빠른 수렴
# )

# # 학습 완료 후 모델 저장
# model.save('model_8_s_100_16_960.pt')

In [30]:
# # # 모델 평가
# metrics = model.val()

In [31]:
import yaml

# YAML 파일에 저장할 데이터
yaml_data = {
    "train": "C:/Users/dawoo/desktop/sf7/coding/pytorch/project/train/images",
    "test": "C:/Users/dawoo/desktop/sf7/coding/pytorch/project/test/images",
    "val": "C:/Users/dawoo/desktop/sf7/coding/pytorch/project/valid/images",
    "nc": 5,
    "names": ["ok", "bumper_faulty", "door_faulty", "fender_faulty", "frame_faulty"],
}

# YAML 파일 생성
with open("data.yaml", "w") as yaml_file:
    yaml.dump(yaml_data, yaml_file, default_flow_style=False, sort_keys=False)

print("YAML 파일이 성공적으로 생성되었습니다.")


YAML 파일이 성공적으로 생성되었습니다.


In [32]:
from ultralytics import YOLO
# 모델 로드
model = YOLO("yolov8s.pt")

# 학습 설정
model.train(
    data="data.yaml",  # 데이터 설정 파일 경로
    epochs=200,  # 에폭 수를 늘려서 더 많은 학습 진행
    batch=4,  # 배치 크기 늘리기
    imgsz=640,  # 입력 이미지 크기
    device=0,  # GPU 사용
    workers=3,  # 데이터 로드 병렬 처리 워커 수
    project="C:\\Users\\dawoo\\Desktop\\SF7\\coding\\PYTORCH\\project\\detect",
    name="part_damage_detection",
)

# 학습 완료 후 모델 저장
model.save('model_8_s_200_4_958.pt')

New https://pypi.org/project/ultralytics/8.3.65 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.63  Python-3.12.0 torch-2.5.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
engine\trainer: task=detect, mode=train, model=yolov8s.pt, data=data.yaml, epochs=120, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=True, device=0, workers=4, project=C:\Users\dawoo\Desktop\SF7\coding\PYTORCH\project\detect, name=part_damage_detection, exist_ok=False, pretrained=True, optimizer=Adamw, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, 

train: Scanning C:\Users\dawoo\Desktop\SF7\coding\PYTORCH\project\train\labels... 102 images, 0 backgrounds, 0 corrupt:  11%|█         | 102/960 [00:02<00:20, 42.62it/s]c:\Users\dawoo\Desktop\SF7\coding\PYTORCH\pytorch-env\Lib\site-packages\PIL\Image.py:3186: DecompressionBombWarning: Image size (108000000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
train: Scanning C:\Users\dawoo\Desktop\SF7\coding\PYTORCH\project\train\labels... 960 images, 0 backgrounds, 0 corrupt: 100%|██████████| 960/960 [00:17<00:00, 54.13it/s] 

train: WARNING  C:\Users\dawoo\Desktop\SF7\coding\PYTORCH\project\train\images\204_101_10_037c081e-5304-4903-8ef3-82f691f87d30.jpg: corrupt JPEG restored and saved
train: WARNING  C:\Users\dawoo\Desktop\SF7\coding\PYTORCH\project\train\images\204_101_10_0b5ba85a-ac85-4c67-869b-cf9b0c6fa487.jpg: corrupt JPEG restored and saved
train: WARNING  C:\Users\dawoo\Desktop\SF7\coding\PYTORCH\project\train\images\204_101_10_10c2ed64-da53-41e6-afd5-64d970ad03e3.jpg: corrupt JPEG restored and saved
train: WARNING  C:\Users\dawoo\Desktop\SF7\coding\PYTORCH\project\train\images\204_101_10_1bddf87a-2fec-45ab-9021-c9c6051939e0.jpg: corrupt JPEG restored and saved
train: WARNING  C:\Users\dawoo\Desktop\SF7\coding\PYTORCH\project\train\images\204_101_10_3507f4e0-e4a4-441c-ac77-97ca4cd15e16.jpg: corrupt JPEG restored and saved
train: WARNING  C:\Users\dawoo\Desktop\SF7\coding\PYTORCH\project\train\images\204_101_10_395c646b-88fe-433f-ace8-73cd04ac7fa9.jpg: corrupt JPEG restored and saved
train: WARNING  

train: New cache created: C:\Users\dawoo\Desktop\SF7\coding\PYTORCH\project\train\labels.cache
WARNING  cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


train: Caching images (0.6GB RAM): 100%|██████████| 960/960 [00:25<00:00, 38.10it/s]
val: Scanning C:\Users\dawoo\Desktop\SF7\coding\PYTORCH\project\valid\labels... 240 images, 0 backgrounds, 0 corrupt: 100%|██████████| 240/240 [00:05<00:00, 41.84it/s]

...

    120/120       2.1G      1.716      1.845      1.829         13        640: 100%|██████████| 120/120 [00:25<00:00,  4.76it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 15/15 [00:02<00:00,  5.83it/s]

                   all        240        293     0.0607     0.0809     0.0229    0.00842



120 epochs completed in 0.954 hours.
Optimizer stripped from C:\Users\dawoo\Desktop\SF7\coding\PYTORCH\project\detect\part_damage_detection\weights\last.pt, 22.5MB
Optimizer stripped from C:\Users\dawoo\Desktop\SF7\coding\PYTORCH\project\detect\part_damage_detection\weights\best.pt, 22.5MB

Validating C:\Users\dawoo\Desktop\SF7\coding\PYTORCH\project\detect\part_damage_detection\weights\best.pt...
Ultralytics 8.3.63  Python-3.12.0 torch-2.5.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
Model summary (fused): 168 layers, 11,127,519 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 15/15 [00:02<00:00,  5.71it/s]


                   all        240        293      0.059     0.0647     0.0221    0.00935
                    ok         48         48          0          0          0          0
         bumper_faulty         53         64     0.0627      0.125     0.0245    0.00997
           door_faulty         51         61      0.111     0.0492     0.0499     0.0232
         fender_faulty         40         57     0.0792     0.0702     0.0209    0.00818
          frame_faulty         48         63     0.0423     0.0794     0.0151    0.00541
Speed: 0.5ms preprocess, 4.9ms inference, 0.0ms loss, 1.8ms postprocess per image
Results saved to C:\Users\dawoo\Desktop\SF7\coding\PYTORCH\project\detect\part_damage_detection


In [39]:
from ultralytics import YOLO
# 모델 로드 및 추론
model = YOLO("C:\\Users\\dawoo\\desktop\\sf7\\coding\\pytorch\\project\\model_8_s_120_8_958.pt")
results = model.predict(source="C:\\Users\\dawoo\\desktop\\sf7\\coding\\pytorch\\project\\test\\images", save=True, imgsz=640)


image 1/300 C:\Users\dawoo\desktop\sf7\coding\pytorch\project\test\images\204_101_10_156cdb63-215b-4855-96b6-67fbc146b609.jpg: 320x640 (no detections), 75.1ms
image 2/300 C:\Users\dawoo\desktop\sf7\coding\pytorch\project\test\images\204_101_10_1a001aec-998d-47a9-b5df-a5fcd7b164ae.jpg: 320x640 (no detections), 11.0ms
image 3/300 C:\Users\dawoo\desktop\sf7\coding\pytorch\project\test\images\204_101_10_1f13f786-cf69-425d-9a8b-0476c0193387.jpg: 288x640 (no detections), 65.0ms
image 4/300 C:\Users\dawoo\desktop\sf7\coding\pytorch\project\test\images\204_101_10_33c0ce7c-7fad-4364-8954-e5f55fca837c.jpg: 320x640 (no detections), 18.0ms
image 5/300 C:\Users\dawoo\desktop\sf7\coding\pytorch\project\test\images\204_101_10_37ed497f-91c0-4163-b145-7392934fe398.jpg: 288x640 (no detections), 12.9ms
image 6/300 C:\Users\dawoo\desktop\sf7\coding\pytorch\project\test\images\204_101_10_4e15e852-4f0c-4eb3-9382-785943894c8c.jpg: 480x640 (no detections), 62.4ms
image 7/300 C:\Users\dawoo\desktop\sf7\coding

In [34]:
# count = 0
# for result in results:
#     # 바운딩 박스가 없는 경우 'no detections'으로 간주
#     if result.boxes is None or len(result.boxes.xywh) == 0:
#         count += 1

# print(f"no detections : {count}")
# print(f"len(results) : {len(results)}")


In [44]:
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt

# 학습된 모델 경로
model = YOLO("model_8_s_120_8_958.pt")  # 예: 'model_8_m_100_8_640.pt'

# 이미지 파일 경로
image_path = "C:\\Users\\dawoo\\Desktop\\범퍼2.jpg"

# 이미지를 불러와서 예측
result = model(image_path)

# 예측 결과 출력
result[0].show()  # 첫 번째 예측 결과의 바운딩 박스를 이미지로 출력

# 예측된 객체들의 정보 출력
boxes = result[0].boxes.xywh  # 바운딩 박스 정보 (x_center, y_center, width, height)
print(f"boxes : {boxes}")  # 예측된 바운딩 박스 정보 출력

# 만약 이미지 저장을 원한다면:
result[0].save("C:\\Users\\dawoo\\Desktop\\predict.jpg")  # 예측된 이미지를 저장



image 1/1 C:\Users\dawoo\Desktop\2.jpg: 448x640 (no detections), 17.0ms
Speed: 2.0ms preprocess, 17.0ms inference, 1.0ms postprocess per image at shape (1, 3, 448, 640)
boxes : tensor([], device='cuda:0', size=(0, 4))


'C:\\Users\\dawoo\\Desktop\\predict.jpg'

In [40]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, roc_curve, roc_auc_score
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

pred = []
test_test_20 = pd.concat([test_data_ok, test_data_fault], ignore_index=True)

for result in results:
    if len(result.boxes) > 0:  # Detection이 있을 경우
        # 각 box에서 confidence가 가장 높은 클래스 선택
        pred.append(int(result.boxes[0].cls))
    else:
        pred.append(0)  # No Detection은 '0'으로 처리


# y_test 생성
y_test = []

for i in range(len(test_test_20)):
    if test_test_20.iloc[i, 1] == 1:  # 열 1에 접근
        y_test.append(0)
    elif test_test_20.iloc[i, 2] == '범퍼':  # 열 2에 접근
        y_test.append(1)
    elif test_test_20.iloc[i, 2] == '도어':
        y_test.append(2)
    elif test_test_20.iloc[i, 2] == '휀더':
        y_test.append(3)
    elif test_test_20.iloc[i, 2] == '프레임':
        y_test.append(4)


# data evaluation


# 위의 ROC 커브 코드 재사용 가능

print(y_test)
print(pred)

# 혼동 행렬
confmat = confusion_matrix(y_true=y_test, y_pred=pred)
print('혼동 행렬: ', confmat, sep='\n')


# 정확도
accuracy = accuracy_score(y_true=y_test, y_pred=pred)
print('정확도:', accuracy)

# 정밀도(precision)
precision = precision_score(y_true=y_test, y_pred=pred, average='macro')
print('정밀도: ', precision)


# 재현율(recall)
recall = recall_score(y_true=y_test, y_pred=pred, average='macro')
print('재현율: ', recall)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 4, 3, 4, 4, 2, 4, 2, 3, 3, 3, 4, 4, 3, 4, 2, 3, 1, 2, 1, 1, 3, 2, 4, 4, 4, 4, 2, 1, 1, 1, 1, 2, 3, 4, 1, 2, 1, 1, 4, 2, 3, 1, 4, 1, 4, 2, 2, 4, 3, 1, 3, 1, 3, 2, 1, 1, 3, 2, 4, 3, 1, 3, 1, 3, 1, 3, 2, 3, 4, 3, 2, 2, 1, 3, 2, 1, 3, 2, 4, 1, 1, 4, 1, 1, 3, 1, 4, 3, 4, 2, 2, 4, 4, 1, 4, 4, 2, 4, 3, 2, 4, 2, 4, 3, 2, 4, 3, 2, 1, 4, 3, 3, 1, 4, 1, 2, 4, 2, 2, 1, 4, 2, 2, 3, 4, 3, 1, 2, 1, 2, 3, 4, 1, 3, 1, 4, 1, 2, 1, 4, 2, 2, 2, 1, 4, 3, 4, 3, 3, 4, 3, 1, 4, 4, 1, 1, 2, 2, 1, 4, 1, 2, 1, 2, 1, 2, 1, 1, 4, 3, 1, 2, 1, 3, 2, 2, 3, 3, 2, 4, 2, 2, 4, 1, 2, 4, 1, 2, 1, 2, 3, 1, 4, 1, 4, 1, 1, 1, 3, 4, 3, 4, 2, 3, 3, 3, 2, 4, 4, 4, 4, 3, 1, 4, 2, 4, 1, 4, 2, 3, 1, 3, 4, 4, 2, 1, 1, 2, 2, 1, 3, 2, 1, 3, 4, 3, 1, 3]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

c:\Users\dawoo\Desktop\SF7\coding\PYTORCH\pytorch-env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [37]:
# print("results 길이:", len(results))
# print("test_test_20 길이:", len(test_test_20))


In [38]:
# import matplotlib.pyplot as plt
# from sklearn.metrics import (
#     accuracy_score, precision_score, recall_score,
#     confusion_matrix, classification_report
# )
# import pandas as pd
# import numpy as np

# pred = []

# test_test_20 = pd.concat([test_data_ok, test_data_fault], ignore_index=True)

# for result in results:
#     if hasattr(result, 'boxes') and len(result.boxes) > 0:  # Detection이 있을 경우
#         pred.append(int(result.boxes[0].cls))
#     else:
#         pred.append(0)  # No Detection은 '0'으로 처리

# # y_test 생성
# y_test = []
# for i in range(len(test_test_20)):
#     if test_test_20.iloc[i, 1] == 1:
#         y_test.append(0)
#     elif str(test_test_20.iloc[i, 2]) == '범퍼':
#         y_test.append(1)
#     elif str(test_test_20.iloc[i, 2]) == '도어':
#         y_test.append(2)
#     elif str(test_test_20.iloc[i, 2]) == '휀더':
#         y_test.append(3)
#     elif str(test_test_20.iloc[i, 2]) == '프레임':
#         y_test.append(4)

# # 데이터 평가
# assert len(y_test) == len(pred), "y_test와 pred의 길이가 일치하지 않습니다."

# print("y_test:", y_test)
# print("pred:", pred)

# # 혼동 행렬
# confmat = confusion_matrix(y_true=y_test, y_pred=pred)
# print('혼동 행렬:')
# print(confmat)

# # 성능 지표
# accuracy = accuracy_score(y_true=y_test, y_pred=pred)
# print('정확도:', accuracy)

# precision = precision_score(y_true=y_test, y_pred=pred, average='macro')
# print('정밀도:', precision)

# recall = recall_score(y_true=y_test, y_pred=pred, average='macro')
# print('재현율:', recall)

# print("Classification Report:")
# print(classification_report(y_true=y_test, y_pred=pred))


NameError: name 'results' is not defined

In [ ]:
import cv2
from ultralytics import YOLO

# YOLOv8 모델 로드
model_path = "C:\\Users\\dawoo\\desktop\\sf7\\coding\\pytorch\\project\\part_damage_detection.pt"  # 학습한 모델 경로
model = YOLO(model_path)
model.to("cuda")

# 카메라 열기
cap = cv2.VideoCapture(0)  # 기본 카메라 (ID 0)
if not cap.isOpened():
    print("카메라를 열 수 없습니다.")
    exit()

# 카메라 프레임 크기 설정 (선택 사항)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

# 색상 설정 (바운딩 박스와 텍스트)
box_color = (0, 255, 0)  # 녹색
text_color = (255, 255, 255)  # 흰색

print("Press 'q' to exit...")

while True:
    # 프레임 읽기
    ret, frame = cap.read()
    if not ret:
        print("프레임을 읽을 수 없습니다.")
        break
    flip_frame = cv2.flip(frame, 1)
    # 모델 추론 (YOLOv8)
    results = model.predict(source=frame, conf=0.5, show=False,
                    verbose=False)  # 이미지에 대해 추론
    detections = results[0]  # 첫 번째 결과 가져오기

    # 바운딩 박스와 라벨 그리기
    for box in detections.boxes:
        # 바운딩 박스 좌표 (정규화된 값 아님)
        x1, y1, x2, y2 = map(int, box.xyxy[0])  # 좌상단(x1, y1), 우하단(x2, y2)
        conf = box.conf[0]  # 신뢰도
        cls = int(box.cls[0])  # 클래스 ID
        label = f"{model.names[cls]} {conf:.2f}"  # 클래스 이름과 신뢰도

        # 바운딩 박스와 라벨 추가
        cv2.rectangle(frame, (x1, y1), (x2, y2), box_color, 2)  # 바운딩 박스
        cv2.putText(frame, label, (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 2)

    # 프레임 화면에 표시
    cv2.imshow("YOLOv8 Real-Time Detection", flip_frame)

    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 자원 해제
cap.release()
cv2.destroyAllWindows()

## 손실 그래프

In [ ]:
# loss function
import matplotlib.pyplot as plt

# 손실 감소 그래프
plt.subplot(1, 2, 1)
plt.plot(losses)
plt.grid()
plt.title('Loss over Epochs')
plt.xlabel('epoch')
plt.ylabel('Loss')

## 데이터 검증

In [ ]:
## data evaluation

from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, roc_curve, roc_auc_score
pred=model(y_test)


# 혼동 행렬
confmat = confusion_matrix(y_true=y_test, y_pred=pred)
print('혼동 행렬: ', confmat)


# 정확도
accuracy=accuracy_score(y_true=y_test, y_pred=pred)
print('정확도: ', accuracy)


# 정밀도(precision)
precision = precision_score(y_true=y_test, y_pred=pred)
print('정밀도: ', precision)


# 재현율(recall)
recall = recall_score(y_true=y_test, y_pred=pred)
print('재현율: ', recall)


# roc auc
fpr, tpr, _ = roc_curve(y_true=y_test, y_pred=pred)

plt.plot(fpr, tpr)
plt.xlabel('FP Rate')
plt.ylabel('TP Rate')

plt.show()

print('auc 점수: ', roc_auc_score(y_test, pred))